# Data setup for training

In [ ]:
import os

In [ ]:
DATA_DIR = ""

### original 178 category data

In [ ]:
BUCKET_NAME = ""
BASE_DATA_DIR= os.path.normpath(os.path.join(DATA_DIR, ''))

### collect more data version

In [ ]:
BUCKET_NAME = ""
BASE_DATA_DIR= os.path.normpath(os.path.join(DATA_DIR, ''))

### collect 3000 data version

In [ ]:
BUCKET_NAME = ""
BASE_DATA_DIR= os.path.normpath(os.path.join(DATA_DIR, ''))

In [ ]:
%%time
%%capture

if not os.path.isdir(BASE_DATA_DIR):
    os.makedirs(BASE_DATA_DIR) 

    REGION_NAME = "ap-northeast-1"
    DEFAULT_OUTPUT = "text"

    DIST_DATA_DIR = os.path.normpath(BASE_DATA_DIR)
    
    os.environ['AWS_DEFAULT_REGION'] = REGION_NAME
    os.environ['AWS_DEFAULT_OUTPUT'] = DEFAULT_OUTPUT

    print("sync")
    !aws s3 sync {BUCKET_NAME} {DIST_DATA_DIR}
    print("sync done")

### Update some category



In [ ]:
TRAIN_DATA_DIR = os.path.normpath(os.path.join(BASE_DATA_DIR, "train"))
VALID_DATA_DIR = os.path.normpath(os.path.join(BASE_DATA_DIR, "valid"))

In [ ]:
s3base = BUCKET_NAME

In [ ]:
def update_category(updatekey):
    update_train = os.path.join(TRAIN_DATA_DIR, updatekey)
    update_valid = os.path.join(VALID_DATA_DIR, updatekey)
    s3valid = os.path.join(os.path.join(s3base, "valid"), updatekey)
    s3train = os.path.join(os.path.join(s3base, "train"), updatekey)
    ! rm -r {update_train}
    ! rm -r {update_valid}
    ! aws s3 cp {s3train} {update_train} --recursive
    ! aws s3 cp {s3valid} {update_valid} --recursive

In [ ]:
update_category('33')
update_category('129')

### corrupted data cleanup

In [ ]:
from PIL import Image
import glob
import shutil
BROKEN = 'corrupted'

In [ ]:
def move_to_broken(f):
    moveddir = os.path.dirname(os.path.join(BROKEN, f))
    os.path.isdir(moveddir) or os.makedirs(moveddir)
    shutil.copy(f, moveddir)
    os.remove(f)
    print("{} is broken, move it.".format(f))

In [ ]:
# TARGET = 'data'
TARGET = BASE_DATA_DIR


os.makedirs(BROKEN, exist_ok=True)

for f in glob.iglob("{}/**/*.jpg".format(TARGET), recursive=True):
    try:
        with Image.open(f) as dummy:
            continue
    except:
        move_to_broken(f)


## manual remove

In [ ]:
# imread success, but result is not array with shape. I don't know the reason.
f = ''

In [ ]:
move_to_broken(f)

# Prerequisite

- put .pgpass inside container
- put awskeys.txt file in current folder 

In [ ]:
import os
import json

In [ ]:
with open('awskeys.txt', 'r') as outfile:
    dic = json.load(outfile)
AWS_KEY_ID = dic["AWS_KEY_ID"]
AWS_SECRET_KEY = dic["AWS_SECRET_KEY"]



In [ ]:
users = [
]

In [ ]:
from akagi.data_sources import RedshiftDataSource
from cooking_log_image_fetcher import Fetcher
from scipy.misc import imread
import os


def build_condition(users):
    conditions = []

    for row in users:
        conditions.append("""
            (
            )
        """.format(**row))

    return ' or '.join(conditions)


def fetch_cooking_log_thumbnail(users):
    with RedshiftDataSource.for_query(
        """
        """.format(condition=build_condition(users)),
        ) as ds:

        for row in ds:
            upload_id, _, user_id = int(row[0]), int(row[1]), int(row[2])
            body = Fetcher.fetch_thumbnail(user_id, upload_id, env='')
            image = imread(body)
            yield body, image, upload_id

In [ ]:
assert (AWS_KEY_ID is not None) and (AWS_SECRET_KEY is not None), "SET your keys."
os.environ['AWS_ACCESS_KEY_ID'] = AWS_KEY_ID
os.environ['AWS_SECRET_ACCESS_KEY'] = AWS_SECRET_KEY
os.environ['AWS_DEFAULT_REGION'] = "ap-northeast-1"

In [ ]:
output_dir = ''

In [ ]:
for body, image, upload_id in fetch_cooking_log_thumbnail(users):

    output_filename = "%s/%s.jpg" % (output_dir, upload_id)
    os.makedirs(os.path.dirname(output_filename), exist_ok=True)

    with open(output_filename, 'wb') as f:
        f.write(body.getvalue())

In [ ]:
import glob

len(list(glob.glob(os.path.normpath("{}/*.jpg").format(output_dir))))

# Check data from flicker

In [ ]:
!pip3 install flickrapi

### Need kernel restart here (to use flickerapi)

In [ ]:
import flickrapi

api_key = u''
api_secret = u''

In [ ]:
flickr = flickrapi.FlickrAPI(api_key, api_secret)

In [ ]:
import os, glob
import requests
import itertools
from concurrent import futures

In [ ]:
output_dir=''


In [ ]:
os.makedirs(output_dir, exist_ok=True)

In [ ]:
def download_one(photoid, url, destdir):
    if url is None:
        print("url none. id {}".format(photoid))
        return
    resp = requests.get(url)
    savepath = os.path.join(destdir, os.path.basename("{}.jpg".format(photoid)))
    if not os.path.exists(savepath):
        with open(savepath, 'wb') as fp:
            fp.write(resp.content)

In [ ]:
def chunks(iterable, size):
    iterator = iter(iterable)
    for first in iterator:
        yield itertools.chain([first], itertools.islice(iterator, size - 1))

In [ ]:
# MAX_WORKERS = 5
MAX_WORKERS = 20

In [ ]:
def download_chunk(chunked_pairs, destdir):
    chunk = list(chunked_pairs)
    workers = min(MAX_WORKERS, len(chunk))
    with futures.ThreadPoolExecutor(workers) as executor:
        res = executor.map(lambda pair: download_one(pair[0], pair[1], destdir), chunk)
    return len(list(res))

In [ ]:
import time


In [ ]:
def download_all_chunks(pairgen, destdir, chunksize=500):
    downloadcnt = 0
    for chunk in chunks(pairgen, chunksize):
        lchunk = list(chunk)
        download_chunk(lchunk, destdir)
        downloadcnt += len(lchunk)
        print("{} downloaded".format(downloadcnt))
        time.sleep(5)
        

### food retrieval

In [ ]:
ID=''

In [ ]:
def get_photo_idurls(groupid=ID, **namedargs):
    return ((photo.get("id"), photo.get("url_o")) for photo in flickr.walk(tag_mode='all', group_id=groupid,
                         license='2,3,4,5,6,9',
                         # min_taken_date=begindt,
                        # max_taken_date=enddt,
                         extras='url_o',
                        **namedargs))

In [ ]:
%%time
download_all_chunks(get_photo_idurls(min_taken_date='2016-12-31'), output_dir, 500)

In [ ]:
%%time

download_all_chunks(get_photo_idurls(min_taken_date='2015-12-31', max_taken_date='2017-01-01'), output_dir, 500)

In [ ]:
%%time
download_all_chunks(get_photo_idurls(min_taken_date='2011-12-31', max_taken_date='2016-01-01'), output_dir, 500)

In [ ]:
%%time
download_all_chunks(get_photo_idurls(min_taken_date='2007-12-31', max_taken_date='2012-01-01'), output_dir, 500)

In [ ]:
%%time
download_all_chunks(get_photo_idurls(min_taken_date='2006-12-31', max_taken_date='2008-01-01'), output_dir, 500)

In [ ]:
%%time
download_all_chunks(get_photo_idurls('2011-12-31', '2014-01-01'), output_dir, 500)

In [ ]:
%%time
download_all_chunks(get_photo_idurls('2009-12-31', '2012-01-01'), output_dir, 500)

In [ ]:
%%time
download_all_chunks(get_photo_idurls('2008-12-31', '2010-01-01'), output_dir, 500)

In [ ]:
%%time
download_all_chunks(get_photo_idurls('2005-12-31', '2009-01-01'), output_dir, 500)

In [ ]:
%%time
download_all_chunks(get_photo_idurls('2000-12-31', '2006-01-01'), output_dir, 500)

In [ ]:
%%time
download_all_chunks(get_photo_idurls('1990-12-31', '2001-01-01'), output_dir, 500)

In [ ]:
%%time
download_all_chunks(get_photo_idurls('1800-12-31', '1991-01-01'), output_dir, 500)

### Get tagged data

In [ ]:
gen = ((photo.get("id"), photo.get("url_o")) for photo in flickr.walk(tags='', tag_mode='all',
                         license='2,3,4,5,6,9',
                       min_taken_date='2015-12-31', max_taken_date='2017-01-01',
                         extras='url_o'))

In [ ]:
i = 0

In [ ]:
for pair in gen:
    print(pair)
    i += 1
    if i > 5:
        break

In [ ]:
def get_pasta_idurl(**namedargs):
    return ((photo.get("id"), photo.get("url_o")) for photo in flickr.walk(tag_mode='all', tags='', 
                         license='2,3,4,5,6,9',
                         extras='url_o',
                        **namedargs))

In [ ]:
%%time
download_all_chunks(get_pasta_idurl(min_taken_date='2016-12-31'), output_dir, 500)

In [ ]:
%%time

download_all_chunks(get_pasta_idurl(min_taken_date='2014-12-31', max_taken_date='2017-01-01'), output_dir, 500)

In [ ]:
%%time
download_all_chunks(get_pasta_idurl(min_taken_date='2000-12-31', max_taken_date='2015-01-01'), output_dir, 500)